In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from utils import generate_groups, generate_points_and_distances, compute_minimum_distances, compute_medical_deserts, get_demographics_of_medical_deserts
from facility_location import facility_location, round_lp_solution
import os
from src.usa.states import USAState
from src.usa.constants import state_names
import csv
from tqdm import tqdm
from pprint import pprint
import sys


In [5]:
facility = 'private_schools'
base_path = '../data/usa/new_facilities/' + facility + '/new_facilities_'

filenames = {
    1: base_path + '1.csv',
    2: base_path + '2.csv',
    np.inf: base_path + 'inf.csv'
}

for file in filenames.values():
    with open(file, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['State', 5, 10, 25, 50, 100])

for state in state_names:
    print(state, file=sys.stderr)

    State = USAState(state)
    state_df = State.get_census_data(level='blockgroup')
    existing_distances_label = 'closest_distance_' + facility

    points, urban_points, pairwise_distances, existing_distances = generate_points_and_distances(state_df, existing_distances_label)
    groups = generate_groups(state_df)

    medical_deserts_df = pd.DataFrame(columns=['k', 'p','n_deserts', 'white_only', 'black_only', 'other'])

    for p in [1, 2, np.inf]:
        print(f'p = {p}', file=sys.stderr)
        existing_facilities = []
        new_facilities = {}
        cumulative_k = 0

        desert_df = compute_medical_deserts(state_df, poverty_threshold=20, n_urban=2, n_rural=10, distance_label=existing_distances_label)
        white_only, black_only, hispanic, other = get_demographics_of_medical_deserts(desert_df)
        print(0, len(desert_df), white_only, black_only, hispanic, other, file=sys.stderr)

        for k in [5, 5, 15, 25, 50]:
            cumulative_k += k
            W, X, Y, Z, model = facility_location(points=points,
                                                  urban_points=urban_points,
                                                  groups=groups,
                                                  pairwise_distances=pairwise_distances,
                                                  existing_facilities=existing_facilities,
                                                  existing_distances=existing_distances,
                                                  k=k,
                                                  p=p)
            new_facilities[cumulative_k] = list(round_lp_solution(points=points, X=X, Y=Y, pairwise_distances=pairwise_distances, k=k, existing_facilities=existing_facilities, alpha=0.50))
            existing_facilities = existing_facilities + new_facilities[cumulative_k]
            if 'existing' in existing_facilities:
                existing_facilities.remove('existing')

            new_distances_label = existing_distances_label + '_chaincompletion_' + str(cumulative_k)
            state_df = compute_minimum_distances(state_df, existing_facilities, existing_distances_label, new_distances_label)
            desert_df = compute_medical_deserts(state_df, poverty_threshold=20, n_urban=2, n_rural=10, distance_label=new_distances_label)

            white_only, black_only, hispanic, other = get_demographics_of_medical_deserts(desert_df)
            print(cumulative_k, len(desert_df), white_only, black_only, hispanic, other, file=sys.stderr)

        with open(filenames[p], 'a') as file:
            writer = csv.writer(file)
            writer.writerow([state, new_facilities[5],
                             new_facilities[5] + new_facilities[10],
                             new_facilities[5] + new_facilities[10] + new_facilities[25],
                             new_facilities[5] + new_facilities[10] + new_facilities[25] + new_facilities[50],
                             new_facilities[5] + new_facilities[10] + new_facilities[25] + new_facilities[50] + new_facilities[100]])


Alabama
100%|██████████| 3915/3915 [00:02<00:00, 1825.91it/s]
p = 1
0 423 251 138 5 29
5 394 231 136 2 25
10 376 223 128 1 24
25 336 207 111 1 17
50 264 164 88 1 11
100 189 123 63 1 2
p = 2
0 423 251 138 5 29
5 394 229 136 4 25
10 374 222 128 1 23
25 324 191 114 1 18
50 253 156 85 1 11
100 167 110 53 1 3
p = inf
0 423 251 138 5 29
5 396 228 136 5 27
10 369 205 136 3 25
25 317 161 129 3 24
50 270 133 110 3 24
100 180 81 82 1 16
Alaska
100%|██████████| 503/503 [00:00<00:00, 10160.98it/s]
p = 1
0 39 9 0 0 30
5 36 9 0 0 27
10 33 8 0 0 25
25 22 4 0 0 18
50 7 2 0 0 5
100 0 0 0 0 0
p = 2
0 39 9 0 0 30
5 36 9 0 0 27
10 32 8 0 0 24
25 19 5 0 0 14
50 8 2 0 0 6
100 1 0 0 0 1
p = inf
0 39 9 0 0 30
5 35 9 0 0 26
10 31 9 0 0 22
25 19 8 0 0 11
50 13 6 0 0 7
100 7 4 0 0 3
Arizona
100%|██████████| 4729/4729 [00:03<00:00, 1415.87it/s]
p = 1
0 437 151 0 166 120
5 413 136 0 158 119
10 401 131 0 154 116
25 320 119 0 103 98
50 268 96 0 88 84
100 172 68 0 34 70
p = 2
0 437 151 0 166 120
5 411 134 0 160 117
1